## 🚀 Getting started with your own PyPSA Model  

### 🎯 Objective

By the end of this session, you will be able to:

- Understand how PyPSA v0.34+ supports importing models from Excel files using component-named sheets.
- Use Excel as a simple and accessible entry point for building PyPSA models.
- Explore and adapt two resources provided in this repository:
  - A blank Excel template to begin building your own model.
  - A worked example of a simplified South African power system using 9 nodes and interconnecting links.

Additionally, you'll gain awareness of best practices for scaling up to more complex models using reproducible workflows.



### ✍️ Authors
<div style="display: flex; align-items: center; justify-content: space-between; flex-wrap: nowrap; gap: 2rem;">

  <div style="flex: 1; min-width: 250px;">
      <a href="https://www.linkedin.com/in/virio-andreyana" target="_blank">Virio Andreyana</a><br>
      <a href="https://www.linkedin.com/in/andreas-denyer" target="_blank">Andreas Denyer</a><br>
      <a href="https://www.linkedin.com/in/priyeshgosai" target="_blank">Priyesh Gosai</a>
    </p>
  </div>

  <div style="flex-shrink: 0;">
    <a href="https://openenergytransition.org/index.html" target="_blank">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="Open Energy Transition logo">
    </a>
  </div>

</div>


### ⚙️ Setup Environment

**💻 Running in Google Colab**

This notebook is designed to run in **Google Colab**, which provides access to a cloud-based virtual machine. While this is convenient and requires no local setup, it's important to note that **Colab sessions are temporary** — all files and installations are **lost when the session ends**.

To ensure your work is preserved:

- The notebook is set up to **link with your Google Drive**. This allows you to save input files, results, and any model modifications to your own Drive for later use.

- The notebook is also linked to a **GitHub repository**. You can choose whether or not to **pull the latest version of the repository** each time you run the notebook using the boolean options provided below.

  - If enabled, the notebook will:
    - Pull the latest version of the repository into your Drive.
    - Create a backup folder containing your existing work, timestamped for traceability.
    - To avoid cluttering your Google Drive, remember to delete old backups or set the `backup_old_repo = False` option.

Another consideration when using Colab is that **PyPSA and several required packages are not pre-installed** in the Colab environment. These packages must be installed manually each time you connect to a new VM.

For both the GitHub sync and the package installation process, we’ve provided boolean flags that let you **skip these steps** if desired. This gives you more control over the setup process, especially when restarting work or debugging.



**Setup Environment**
1. Provide the path to the repository for this project.
2. Confirm if this is the first run in the Google Colab environment.
3. Confirm if you want to update the repository with the latest changes.

In [ ]:
repo_path  = f'https://github.com/open-energy-transition/EIS-2025'
first_run = True # Set to True if this is the first run of the in the Google Colab environment
update_repo = True # Set to True if you want to update the repository with the latest changes

In [ ]:
# @title Connect this Notebook to your Google Drive
# @markdown Run this cell to link this notebook to the GitHub repository. <br>
# @markdown If the repository already exists, it will back up any modified files and force pull the latest changes.
from google.colab import drive
import os
import subprocess
import shutil
from datetime import datetime
import urllib.request

# Mount Google Drive
drive.mount('/content/drive')

# Base directory
base_dir = '/content/drive/MyDrive/'
os.chdir(base_dir)

FOLDER = os.path.basename(repo_path)


if update_repo:


    # Get timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_folder = f'Backup_{timestamp}'
    backup_path = os.path.join(base_dir, backup_folder)

    # If repo exists, back up and force pull
    if os.path.exists(repo_path):
        os.chdir(repo_path)

        # Get list of modified and untracked files
        result = subprocess.run(['git', 'status', '--porcelain'], stdout=subprocess.PIPE, text=True)
        changed_files = [line[3:] for line in result.stdout.splitlines() if line and line[0] in ('M', 'A', '??')]

        if changed_files:
            os.makedirs(backup_path, exist_ok=True)
            print(f"Backing up {len(changed_files)} changed files to {backup_folder}...")
            for file in changed_files:
                src = os.path.join(repo_path, file)
                dst = os.path.join(backup_path, file)
                os.makedirs(os.path.dirname(dst), exist_ok=True)
                if os.path.exists(src):
                    shutil.copy2(src, dst)

        # Force pull: reset local repo to match remote
        subprocess.run(['git', 'fetch', 'origin'])
        subprocess.run(['git', 'reset', '--hard', 'origin/main'])

    else:
        os.chdir(base_dir)
        subprocess.run(['git', 'clone', repo_path])


    files = {
        "network_blank.xlsx": "17haq369Y7-5JFRjESfDbUUljVyIEUS4u",  # file ID
        "network_south_africa.xlsx": "1hBqZjotxDuZOQb-kkC0JePbc3gxCGcwq",  # file ID
    }

    for name, file_id in files.items():
        print(f"Retrieving {name} from Google Drive")
        url = f"https://docs.google.com/uc?export=download&id={file_id}"
        urllib.request.urlretrieve(url, name)

    print("Current working directory:", os.getcwd())

In [ ]:
# @title Install the required packages
# @markdown Run this cell to install the necessary Python packages for the project.
import subprocess, sys, importlib

try:
    from importlib import metadata  # Python 3.8+
except ImportError:
    import importlib_metadata as metadata  # Backport for older versions

packages = [
    "pypsa",        # power-system modelling & optimization toolbox
    "pypsa[excel]", # pypsa with Excel I/O support
    "folium",       # interactive leaflet-based maps in Python
    "mapclassify"   # spatial data classification for choropleth maps
]

# Install packages
subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", *packages],
    check=True
)

# Check and report installed versions
base_names = [pkg.split("[")[0] for pkg in packages]
missing = [p for p in base_names if importlib.util.find_spec(p) is None]

if not missing:
    print("✅ All packages installed successfully.\n")
    for pkg in base_names:
        try:
            version = metadata.version(pkg)
            print(f"📦 {pkg} version {version}")
        except metadata.PackageNotFoundError:
            print(f"⚠️ {pkg} is not found in metadata.")
else:
    print(f"❌ Missing: {', '.join(missing)}")


To import the excel file


```python
network_name = pypsa.Network('network_name.xlsx')

```

Explore the South African Example

In [ ]:
import pypsa

In [ ]:
network_za = pypsa.Network('network_south_africa.xlsx')

In [ ]:
network_za.loads

In [ ]:
network_za.buses # and similarly for other components

In [ ]:
network_za.optimize() # This will run the optimization on the network

In [ ]:

network_za.export_to_excel('network_export.xlsx') # Export the network to an Excel file


---